In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import os

2026-01-28 08:20:44.865716: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-01-28 08:20:44.900635: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-28 08:20:45.838109: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [9]:
IMG_SIZE = (224, 224)         
BATCH_SIZE = 16
NUM_CLASSES = 1                
DATA_DIR = "/home/legramante/Documentos/cat-and-dogs_Machine-Learning/cats_and_dogs"

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode='nearest'
)

In [11]:
val_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(
    os.path.join(DATA_DIR, "train"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 17500 images belonging to 2 classes.


In [13]:
val_generator = val_datagen.flow_from_directory(
    os.path.join(DATA_DIR, "val"),
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 3750 images belonging to 2 classes.


In [14]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = False 

2026-01-28 08:22:39.685499: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [15]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # regularização
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='sigmoid')(x)

In [16]:
model = Model(inputs=base_model.input, outputs=predictions)

In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [21]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)

In [14]:
history_phase1 = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,  
    callbacks=[early_stop, reduce_lr]
)

Epoch 1/10


2026-01-19 16:25:28.706039: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51380224 exceeds 10% of free system memory.
2026-01-19 16:25:28.755758: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 53231616 exceeds 10% of free system memory.
2026-01-19 16:25:28.782026: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51380224 exceeds 10% of free system memory.
2026-01-19 16:25:28.826440: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51380224 exceeds 10% of free system memory.
2026-01-19 16:25:28.896339: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51380224 exceeds 10% of free system memory.


1094/1094 ━━━━━━━━━━━━━━━━━━━━ 791s 719ms/step - accuracy: 0.5329 - loss: 0.7005 - val_accuracy: 0.5717 - val_loss: 0.6790 - learning_rate: 0.0010
Epoch 2/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 908s 830ms/step - accuracy: 0.5563 - loss: 0.6822 - val_accuracy: 0.6048 - val_loss: 0.6685 - learning_rate: 0.0010
Epoch 3/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 867s 793ms/step - accuracy: 0.5658 - loss: 0.6777 - val_accuracy: 0.6123 - val_loss: 0.6622 - learning_rate: 0.0010
Epoch 4/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 812s 742ms/step - accuracy: 0.5756 - loss: 0.6742 - val_accuracy: 0.6128 - val_loss: 0.6613 - learning_rate: 0.0010
Epoch 5/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 817s 747ms/step - accuracy: 0.5769 - loss: 0.6747 - val_accuracy: 0.6160 - val_loss: 0.6527 - learning_rate: 0.0010
Epoch 6/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 780s 713ms/step - accuracy: 0.5839 - loss: 0.6718 - val_accuracy: 0.6176 - val_loss: 0.6602 - learning_rate: 0.0010
Epoch 7/10
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 865s 791ms/step - acc

In [28]:
model.save('/home/legramante/Documentos/cat-and-dogs_Machine-Learning/models/cat_dog_classifier3.h5')